In [23]:
import numpy as np
import plotly.graph_objects as go
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.stats import norm 

In [24]:
def black_box_function(x, y):
    return np.sin(x*np.cos(x)**2 + y**2)

In [25]:
bounds = [(-2, 2), (-2, 2)]

In [26]:
X_init = np.array([[-1.5, -1.5], [-1.5, -0.5], [-1.5, 0.5], [-1.5, 1.5], [-0.5, -1.5], [-0.5, -0.5], [-0.5, 0.5], [-0.5, 1.5], [0.5, -1.5], [0.5, -0.5]])
y_init = black_box_function(X_init[:, 0], X_init[:, 1])

In [27]:
# Define the Gaussian Process regressor with an RBF kernel
kernel = C(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2))
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

# Fit the Gaussian Process model
gp.fit(X_init, y_init)

GaussianProcessRegressor(kernel=1**2 * RBF(length_scale=1),
                         n_restarts_optimizer=10)

In [28]:
# Define the acquisition function (expected improvement)
def acquisition_function(x):
    x = np.atleast_2d(x)  # Ensure x is 2D array
    mean, std = gp.predict(x, return_std=True)
    best_observed = np.min(y_init)
    gamma = (best_observed - mean) / std
    expected_improvement = std * (gamma * norm.cdf(gamma) + norm.pdf(gamma))
    return -expected_improvement  # maximize expected improvement


In [29]:
# Function to update the GP model with new data points
def update_gp(new_X, new_y):
    global X_init, y_init, gp
    X_init = np.vstack((X_init, new_X))
    y_init = np.concatenate((y_init, new_y))
    gp.fit(X_init, y_init)

In [30]:
def plot_functions():
    x_range = np.linspace(bounds[0][0], bounds[0][1], 50)
    y_range = np.linspace(bounds[1][0], bounds[1][1], 50)
    X, Y = np.meshgrid(x_range, y_range)
    Z_true = black_box_function(X, Y)
    
    # Plot the predicted function
    Z_pred = gp.predict(np.vstack((X.ravel(), Y.ravel())).T).reshape(X.shape)
    fig = go.Figure(data=[go.Surface(z=Z_pred, x=X, y=Y, opacity=0.6)])
    
    # Plot the data points
    fig.add_trace(go.Scatter3d(x=X_init[:, 0], y=X_init[:, 1], z=y_init, mode='markers', marker=dict(size=5), name='Data Points'))
    
    # Compute and plot the 95% confidence interval
    mean, std = gp.predict(np.vstack((X.ravel(), Y.ravel())).T, return_std=True)
    lower_bound = mean - 1.96 * std  # 95% confidence interval
    upper_bound = mean + 1.96 * std
    fig.add_trace(go.Surface(z=lower_bound, x=X, y=Y, opacity=0.05, showscale=False))
    fig.add_trace(go.Surface(z=upper_bound, x=X, y=Y, opacity=0.05, showscale=False))
    
    # Set labels and title
    fig.update_layout(
        scene=dict(xaxis_title='X', yaxis_title='Y',
                    zaxis_title='Output'),
                height=1200,
                title='Gaussian Process Regression'
                )
    
    fig.show()

In [31]:
plot_functions()